In [33]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_excel('~D:\W\Sportbank\data.xlsm')

In [5]:
data.head()

,id,LastLog_days_ago,LastPayment_days_ago,Sessions,Payments,Money
0,1,19,842,21,1,1.00
1,2,2,4,489,25,27.77
2,3,18,334,5,1,4.99
3,4,19,145,512,2,2.98
4,5,2,103,327,4,61.97


In [4]:
#check if dataset has Nan values
data.isna().sum()

id                      0
LastLog_days_ago        0
LastPayment_days_ago    0
Sessions                0
Payments                0
Money                   0
dtype: int64

In [16]:
#select data that is needed for RFM-segmentation
RFM_data = data.drop(['LastLog_days_ago', 'Sessions'], axis=1)\
                .rename(columns={'id':'user_id', 'LastPayment_days_ago':'recency', 'Payments':'frequency', 'Money':'monetary'})
RFM_data.head(5)

,user_id,recency,frequency,monetary
0,1,842,1,1.00
1,2,4,25,27.77
2,3,334,1,4.99
3,4,145,2,2.98
4,5,103,4,61.97


In [6]:
#descriptive statistic for recency value
RFM_data.recency.describe()

count    6344.000000
mean      130.865858
std       130.814596
min         1.000000
25%        29.000000
50%        87.000000
75%       205.000000
max       903.000000
Name: recency, dtype: float64

In [7]:
#descriptive statistic for frequency value
RFM_data.frequency.describe()

count    6344.000000
mean        9.423707
std        19.127101
min         1.000000
25%         2.000000
50%         4.000000
75%         9.000000
max       435.000000
Name: frequency, dtype: float64

In [8]:
#descriptive statistic for monetary value
RFM_data.monetary.describe()

count     6344.000000
mean       115.328695
std        537.277480
min          0.990000
25%          3.980000
50%         14.980000
75%         60.872500
max      24194.420000
Name: monetary, dtype: float64

In [ ]:
# as we haven't well-defined business task of how much segments and which diapason of values each feature(R, F, or M)
# must have, we can try split data into 2 or 3 quantiles and choose that choice where we have intervals which can
# informatively tell about each group. 
# for example, if split Frequency feature group into 3 quantiles, we have not so informative layers between min-Q1 and Q1-Q2
#min - 1.0
#25% - 2.0
#50% - 4.0
#75% - 9.0
# We get insignificant difference range. But if we split data into 3 quantiles, we get clear 3 user groups: 
#min - 1.0
#33% - 2.0
#67% - 7.0

# So I decide split each group into 3 quantiles and get such intervals that describe 3 groups into 3 dimensions

# For Revency dimension:
# 1 - [1, 43] - users that make transactions recently
# 2 - [43, 159] - users that make transactions not so recently
# 3 - [159, 903] - users that make transactions a long time ago

# For Frequency dimension:
# 1 - [1, 2] - users that make single orders
# 2 - [3, 7] - users that make purchase not so often
# 2 - [8, 435] - users that make purchase often

# For Monetary dimension:
# 1 - [0.99, 5.98] - users with a small amount of purchases;
# 2 - [5.99, 38] - users with an average amount of purchases;
# 3 - [38.1, 24195] - users with a large amount of purchases;

In [9]:
quantiles = []
quantiles.append({'feature':'R', 'value':RFM_data.recency.quantile(q=[0.33, 0.67]).to_dict()})
quantiles.append({'feature':'F', 'value':RFM_data.frequency.quantile(q=[0.33, 0.67]).to_dict()})
quantiles.append({'feature':'M', 'value':RFM_data.monetary.quantile(q=[0.33, 0.67]).to_dict()})
quantiles

[{'feature': 'R', 'value': {0.33: 43.0, 0.67: 159.0}},
 {'feature': 'F', 'value': {0.33: 2.0, 0.67: 7.0}},
 {'feature': 'M', 'value': {0.33: 5.98, 0.67: 37.9181}}]

In [21]:
# now when we defined intervals for each group, assign each user a group number for each dimension and define his segment

def define_user_group(column):
    conditions = [
        (RFM_data[column] <= RFM_data[column].quantile(0.33)),
        ((RFM_data[column] > RFM_data[column].quantile(0.33)) & (RFM_data[column] <= RFM_data[column].quantile(0.67))),
        (RFM_data[column] > RFM_data[column].quantile(0.67))]
         
    return conditions
    
    
RFM_data['R'] = np.select(define_user_group('recency'), ['1', '2', '3'])
RFM_data['F'] = np.select(define_user_group('frequency'), ['1', '2', '3'])
RFM_data['M'] = np.select(define_user_group('monetary'), ['1', '2', '3'])

RFM_data['segment'] = RFM_data.R + RFM_data.F + RFM_data.M
RFM_data.head(10)


,user_id,recency,frequency,monetary,R,F,M,segment
0,1,842,1,1.00,3,1,1,311
1,2,4,25,27.77,1,3,2,132
2,3,334,1,4.99,3,1,1,311
3,4,145,2,2.98,2,1,1,211
4,5,103,4,61.97,2,2,3,223
5,6,9,21,113.81,1,3,3,133
6,7,200,2,1.98,3,1,1,311
7,8,90,7,19.99,2,2,2,222
8,9,253,5,9.95,3,2,2,322
9,10,12,9,32.90,1,3,2,132


In [36]:
RFM_data.to_csv(r'D:\W\Sportbank\processed_data.csv', index = False, header=True)